In [1]:
import pandas as pd

import numpy as np

from datetime import date 
from datetime import datetime, timedelta
import sys

import os
from pathlib import Path

import time

import re

import shutil

from os import listdir
from os.path import isfile, join

In [2]:
import sys 
#!{sys.executable} -m pip install --user matplotlib

**Read sites list with ratings**

In [3]:
sites_list_df = pd.read_csv('iq_sites_list_rated(updated).csv')
sites_list_df.head(1)

,news_site,url,rate,Unnamed: 3
0,creteplus.gr,https://creteplus.gr,high,NaN


**Define column names**

In [4]:
header_names = ["type_s", # τύπος social
                "url_s", # url του social
                "external_link_s", # url του άρθρου
                "source_s", # όνομα του social
                "timestamp_dt", # ημερομηνία δημοσίευσης στο social
                "text_txt_el", # κείμενο στο social
                "retweet_count_l", # πλήθος retweets (twitter)
                "followers_i", # πλήθος followers (twitter)
                "following_i", # πλήθος following (twitter)
                "mentions_i", # πλήθος αναφορών (twitter)
                "comments_i", # πλήθος σχολίων (fb)
                "likes_i", # πλήθος likes (fb
                "full_artcl_text" # κείμενο άρθρου στο site (δεν περιλαμβάνεται ο τίτλος)
               ]

## **Define dataset** 

In [5]:
#main_csvs_path = '0_innews_data_sets/iqcsv/'
#main_csvs_path = '0_innews_data_sets/exports_2017/'
main_csvs_path = '0_innews_data_sets/exports_2022/'

## **Process**

**Make comp dfs for high and low quality sites**

**!IMPORTANT: Δεν υπάρχουν low quality sites στα datasets του 2017-2018 και του 2022-2023**

In [6]:
# create dictionaries for the high and low sites dfs
dfs_high = {}
dfs_low = {}

# read csv, make df, put it in the dictionary
for index, row in sites_list_df.iterrows():
    key = row['news_site']
    csv_name = key + '.csv'
    try:
        site_df = pd.read_csv(main_csvs_path + csv_name, header=None, names=header_names, on_bad_lines='skip')
        site_df['site'] = key
        if row['rate'] == 'high':
            dfs_high[key] = site_df
        elif row['rate'] == 'low':
            dfs_low[key] = site_df
    except:
        pass

In [7]:
# make comp dfs for high and low dfs
for q in ['high', 'low']:
    try:
        if q == 'high':
            for i in range(0, len(dfs_high)-1):
                if i == 0:
                    high_comp_df = pd.concat([dfs_high[list(dfs_high.keys())[i]], dfs_high[list(dfs_high.keys())[i+1]]],
                                            ignore_index=True)
                else:
                    high_comp_df = pd.concat([high_comp_df, dfs_high[list(dfs_high.keys())[i+1]]],
                                            ignore_index=True)
        elif q == 'low':
            for i in range(0, len(dfs_low)-1):
                if i==0:
                    low_comp_df = pd.concat([dfs_low[list(dfs_low.keys())[i]], dfs_low[list(dfs_low.keys())[i+1]]],
                                           ignore_index=True)
                else:
                    low_comp_df = pd.concat([low_comp_df, dfs_low[list(dfs_low.keys())[i+1]]],
                                           ignore_index=True)
    except:
        pass
# high_comp_df.reset_index(drop=True, inplace=True)
# low_comp_df.reset_index(drop=True, inplace=True)

**Add ID column**

In [8]:
try:
    high_comp_df.insert(0, 'in_id', high_comp_df.index)
    low_comp_df.insert(0, 'in_id', low_comp_df.index)
except:
    pass

In [9]:
try:
    print(high_comp_df.shape)
    high_comp_df.tail(1)
except:
    print("no high quality sites")
    pass

(1967485, 15)


In [10]:
try:
    print(low_comp_df.shape)
    low_comp_df.tail(1)
except:
    print("no low quality sites")
    pass

no low quality sites


**Pre-process (correct) data**

In [11]:
high_comp_df.tail()

,in_id,type_s,url_s,external_link_s,source_s,timestamp_dt,text_txt_el,retweet_count_l,followers_i,following_i,mentions_i,comments_i,likes_i,full_artcl_text,site
1967480,1967480,https://twitter.com/PressPublica/status/107981...,https://www.presspublica.gr/toyrkia-vinteo-me-...,Τουρκία: Βίντεο με άνδρες που φέρεται ότι μετα...,PressPublica,2018-12-31T21:17:45Z,Τουρκία: Βίντεο με άνδρες που φέρεται ότι μετα...,0,1396,432,0,0,0.0,"Άνδρες που μεταφέρουν βαλίτσες, μέσα στις οποί...",presspublica.gr
1967481,1967481,https://twitter.com/PressPublica/status/107982...,https://www.presspublica.gr/st-theodorakis-to-...,Στ. Θεοδωράκης: Το 2019 να σκεφτούμε και να πρ...,PressPublica,2018-12-31T21:27:40Z,Στ. Θεοδωράκης: Το 2019 να σκεφτούμε και να πρ...,0,1396,432,0,0,0.0,«Το 2019 μπορεί να είναι μία αληθινά νέα χρονι...,presspublica.gr
1967482,1967482,https://twitter.com/PressPublica/status/107982...,https://www.presspublica.gr/mitsotakis-i-chora...,Μητσοτάκης: Η χώρα δεν χρειάζεται άλλους διχασ...,PressPublica,2018-12-31T21:49:19Z,Μητσοτάκης: Η χώρα δεν χρειάζεται άλλους διχασ...,0,1396,432,0,0,0.0,«Το γύρισμα του χρόνου μάς οδηγεί και σε ένα γ...,presspublica.gr
1967483,1967483,https://twitter.com/PressPublica/status/107982...,https://www.presspublica.gr/tramp-argi-i-apoch...,Τραμπ: «Αργή» η αποχώρηση των Αμερικανών στρατ...,PressPublica,2018-12-31T21:52:09Z,Τραμπ: «Αργή» η αποχώρηση των Αμερικανών στρατ...,0,1396,432,0,0,0.0,Ο πρόεδρος των ΗΠΑ\nΝτόναλντ Τραμπ επιβεβαίωσε...,presspublica.gr
1967484,1967484,https://twitter.com/PressPublica/status/107982...,https://www.presspublica.gr/erntogan-minyma-ap...,Ερντογάν: Μήνυμα αποφασιστικότητας για Αιγαίο ...,PressPublica,2018-12-31T21:55:13Z,Ερντογάν: Μήνυμα αποφασιστικότητας για Αιγαίο ...,0,1396,432,0,0,0.0,Μήνυμα πως η Τουρκία επιδεικνύει την αποφασιστ...,presspublica.gr


In [12]:
high_comp_df.nunique()

in_id              1967485
type_s             1967453
url_s              1378964
external_link_s    1274676
source_s                96
timestamp_dt       1881526
text_txt_el        1872709
retweet_count_l         70
followers_i           1332
following_i             53
mentions_i              32
comments_i             393
likes_i               1462
full_artcl_text    1357518
site                    56
dtype: int64

**Στα csvs του 2017-2018 και του 2022-2023**:
- το type_s πρέπει να είναι τιμή twitter
- η στήλη external_link_s πρέπει να έχει τις τιμές της στήλης url_s
- η στήλη url_s πρέπει να έχει τις τιμές της στήλης type_s

In [13]:
# correct values in columns "type_s", "url_s" and "external_link_s"

In [14]:
# ΑΝ ΤΟ DATASET είναι από το 2027-2018 ή το 2022-2023
if (main_csvs_path == '0_innews_data_sets/exports_2017/') | (main_csvs_path == '0_innews_data_sets/exports_2022/'):
    try:
        # βάλε το περιεχόμενο της στήλης "url_s" στη στήλη "external_link_s"
        high_comp_df.external_link_s = high_comp_df.url_s
        low_comp_df.external_link_s = low_comp_df.url_s
    except:
        pass
    try:
        # βάλε το περιεχόμενο της στήλης "type_s" στη στήλη "external_link_s"
        high_comp_df.url_s = high_comp_df.type_s
        low_comp_df.url_s = low_comp_df.type_s
    except:
        pass
    try:
        # βάλε στο type_s την τιμή "twitter"
        high_comp_df.type_s = 'twitter'
        low_comp_df.type_s = 'twitter'
    except:
        pass
high_comp_df.tail(1)

,in_id,type_s,url_s,external_link_s,source_s,timestamp_dt,text_txt_el,retweet_count_l,followers_i,following_i,mentions_i,comments_i,likes_i,full_artcl_text,site
1967484,1967484,twitter,https://twitter.com/PressPublica/status/107982...,https://www.presspublica.gr/erntogan-minyma-ap...,PressPublica,2018-12-31T21:55:13Z,Ερντογάν: Μήνυμα αποφασιστικότητας για Αιγαίο ...,0,1396,432,0,0,0.0,Μήνυμα πως η Τουρκία επιδεικνύει την αποφασιστ...,presspublica.gr


In [15]:
#replace whitespace values with nan
try:
    high_comp_df.replace(r'^\s*$', np.nan, regex=True, inplace=True)
    low_comp_df.replace(r'^\s*$', np.nan, regex=True, inplace=True)
except:
    pass

In [16]:
high_comp_df.tail(1)

,in_id,type_s,url_s,external_link_s,source_s,timestamp_dt,text_txt_el,retweet_count_l,followers_i,following_i,mentions_i,comments_i,likes_i,full_artcl_text,site
1967484,1967484,twitter,https://twitter.com/PressPublica/status/107982...,https://www.presspublica.gr/erntogan-minyma-ap...,PressPublica,2018-12-31T21:55:13Z,Ερντογάν: Μήνυμα αποφασιστικότητας για Αιγαίο ...,0,1396,432,0,0,0.0,Μήνυμα πως η Τουρκία επιδεικνύει την αποφασιστ...,presspublica.gr


In [17]:
# keep rows with type_s values 'facebook' or 'twitter' only
#if (main_csvs_path == '0_innews_data_sets/iqcsv/'):
try:
    high_comp_df = high_comp_df[(high_comp_df.type_s == 'facebook') | (high_comp_df.type_s == 'twitter')]
    low_comp_df = low_comp_df[(low_comp_df.type_s == 'facebook') | (low_comp_df.type_s == 'twitter')]
except:
    pass
high_comp_df.tail(1)

,in_id,type_s,url_s,external_link_s,source_s,timestamp_dt,text_txt_el,retweet_count_l,followers_i,following_i,mentions_i,comments_i,likes_i,full_artcl_text,site
1967484,1967484,twitter,https://twitter.com/PressPublica/status/107982...,https://www.presspublica.gr/erntogan-minyma-ap...,PressPublica,2018-12-31T21:55:13Z,Ερντογάν: Μήνυμα αποφασιστικότητας για Αιγαίο ...,0,1396,432,0,0,0.0,Μήνυμα πως η Τουρκία επιδεικνύει την αποφασιστ...,presspublica.gr


In [18]:
# remove rows with nan values in any column
try:
    high_comp_df = high_comp_df.dropna(how='any')
    low_comp_df = low_comp_df.dropna(how='any')
except:
    pass

In [19]:
# drop rows with duplicate values in url_s column
try:
    high_comp_df.drop_duplicates('url_s', keep=False, inplace=True)
    low_comp_df.drop_duplicates('url_s', keep=False, inplace=True)
except:
    pass

In [20]:
# convert retweet_count_l, followers_i and following_i to float
try:
    high_comp_df.retweet_count_l = high_comp_df.retweet_count_l.astype(float)
    high_comp_df.followers_i = high_comp_df.followers_i.astype(float)
    high_comp_df.following_i = high_comp_df.following_i.astype(float)
    low_comp_df.retweet_count_l = low_comp_df.retweet_count_l.astype(float)
    low_comp_df.followers_i = low_comp_df.followers_i.astype(float)
    low_comp_df.following_i = low_comp_df.following_i.astype(float)
except:
    pass

**Indentify site name in source_s (ποια posts έχουν γίνει από το ίδιο το μέσο που δημοσίευσε το άρθρο)**

In [21]:
# correct site names
# remove substring after first dot in site names
try:
    high_comp_df['sitename'] = high_comp_df['site'].replace(r'\..*', '', regex=True)
    low_comp_df['sitename'] = low_comp_df['site'].replace(r'\..*', '', regex=True)
except:
    pass

In [22]:
# lower upper case letters in source_s names
try:
    high_comp_df['source_s_lower'] = high_comp_df['source_s'].str.lower()
    low_comp_df['source_s_lower'] = low_comp_df['source_s'].str.lower()
except:
    pass

In [23]:
# find source_s names that start with sitename (HIGH SITES)
try:
    print("unique source_s names:", high_comp_df.sitename.nunique())
    print("identified source_s names:", high_comp_df[high_comp_df.apply(lambda row: row['source_s_lower'].startswith(row['sitename']),
                                          axis=1)].sitename.nunique())
    high_comp_df[high_comp_df.apply(lambda row: row['source_s_lower'].startswith(row['sitename']), axis=1)].head(1)
except:
    pass

unique source_s names: 56
identified source_s names: 47


In [24]:
# find source_s names that start with sitename (LOW SITES)
try:
    print("unique source_s names:", low_comp_df.sitename.nunique())
    print("identified source_s names:", low_comp_df[low_comp_df.apply(lambda row: row['source_s_lower'].
                                                                      startswith(row['sitename']),
                                                                      axis=1)].sitename.nunique())
    low_comp_df[low_comp_df.apply(lambda row: row['source_s_lower'].startswith(row['sitename']),
                                    axis=1)].head(1)
except:
    pass

In [ ]:
# pd.set_option('display.max_colwidth', None)
#pd.reset_option('display')

In [34]:
#matches
# try:
#     low_comp_df[low_comp_df.apply(lambda row: row['source_s_lower'].startswith(row['sitename']),
#                                     axis=1)].groupby('sitename').source_s_lower.unique()
# except:
#     pass
# try:
#     high_comp_df[high_comp_df.apply(lambda row: row['source_s_lower'].startswith(row['sitename']),
#                                     axis=1)].groupby('sitename').source_s_lower.unique()
# except:
#     pass

In [33]:
#Ποια source_s ονόματα σχετίζονται με κάθε site;
# try:
#     print(high_comp_df.groupby('site').source_s.unique())
#     print(low_comp_df.groupby('site').source_s.unique())
# except:
#     pass

In [35]:
false_matches_high = ['capitaloskylo', 'capitalcontrol2', 'left2kolou', 'leftfantasy', 
                      'lefterispatra', 'liberal2402', 'liberalove1', 'starlightpocket', 
                      'starcadell', 'starbuzzgr']
false_matches_low = ['olympiada']

In [36]:
try:
    all_matches_high = high_comp_df[high_comp_df.apply(lambda row: row['source_s_lower'].
                                                       startswith(row['sitename']), axis=1)].source_s_lower.unique()
    # print(list(all_matches_high))
    all_matches_low = low_comp_df[low_comp_df.apply(lambda row: row['source_s_lower'].
                                                    startswith(row['sitename']), axis=1)].source_s_lower.unique()
    # print(list(all_matches_low))
except:
    pass

In [37]:
try:
    true_matches_high = []
    for site in all_matches_high:
        if site not in false_matches_high:
            true_matches_high.append(site)
    # print(true_matches_high)
    true_matches_low = []
    for site in all_matches_low:
        if site not in false_matches_low:
            true_matches_low.append(site)
    print(true_matches_low)
except:
    pass

In [38]:
# correct source_s names
try:
    high_comp_df['source_s_new'] = high_comp_df.apply(lambda x: x['sitename'] if x['source_s_lower'] 
                                                      in true_matches_high else x['source_s_lower'], axis=1)
    low_comp_df['source_s_new'] = low_comp_df.apply(lambda x: x['sitename'] if x['source_s_lower'] 
                                                      in true_matches_low else x['source_s_lower'], axis=1)
except:
    pass

### **Calculations for total counts**

**Nbr of posts on social per article**

In [39]:
# count social posts per article
try:
    high_comp_df['social_posts'] = high_comp_df.groupby(['external_link_s', 'type_s']).in_id.transform('count')
    low_comp_df['social_posts'] = low_comp_df.groupby(['external_link_s', 'type_s']).in_id.transform('count')
except:
    pass

In [40]:
# write facebook counts in separate column
try:
    high_comp_df['facebook_posts'] = high_comp_df.apply(lambda x: 
                                                        x['social_posts'] if x['type_s']=='facebook' else 0, axis=1)
    low_comp_df['facebook_posts'] = low_comp_df.apply(lambda x: 
                                                        x['social_posts'] if x['type_s']=='facebook' else 0, axis=1)
except:
    pass

In [41]:
# write tweets counts in separate column
try:
    high_comp_df['tweets'] = high_comp_df.apply(lambda x: 
                                                        x['social_posts'] if x['type_s']=='twitter' else 0, axis=1)
    low_comp_df['tweets'] = low_comp_df.apply(lambda x: 
                                                        x['social_posts'] if x['type_s']=='twitter' else 0, axis=1)
except:
    pass

**Nbr of retweets per article**

In [42]:
try:
    high_comp_df['retweets_total'] = high_comp_df.groupby('external_link_s').retweet_count_l.transform('sum')
    #high_comp_df[(high_comp_df.type_s=='twitter') & (high_comp_df.retweets_total>0)].head(1)
    low_comp_df['retweets_total'] = low_comp_df.groupby('external_link_s').retweet_count_l.transform('sum')
    #low_comp_df[(low_comp_df.type_s=='twitter') & (low_comp_df.retweets_total>0)].head(1)
except:
    pass

**Nbr of likes per article**

In [43]:
try:
    high_comp_df['likes_total'] = high_comp_df.groupby('external_link_s').likes_i.transform('sum')
    #high_comp_df[(high_comp_df.type_s=='facebook') & (high_comp_df.likes_total>0)].head(1)
    low_comp_df['likes_total'] = low_comp_df.groupby('external_link_s').likes_i.transform('sum')
    #low_comp_df[(low_comp_df.type_s=='facebook') & (low_comp_df.likes_total>0)].head(1)
except:
    pass

**Nbr of comments per article**

In [44]:
try:
    high_comp_df['comments_total'] = high_comp_df.groupby('external_link_s').comments_i.transform('sum')
    #high_comp_df[(high_comp_df.type_s=='facebook') & (high_comp_df.comments_total>0)].head(1)
    low_comp_df['comments_total'] = low_comp_df.groupby('external_link_s').comments_i.transform('sum')
    #low_comp_df[(low_comp_df.type_s=='facebook') & (low_comp_df.comments_total>0)].head(1)
except:
    pass

**Nbr of followers per article**

In [45]:
try:
    high_comp_df['followers_total'] = high_comp_df.groupby('external_link_s').followers_i.transform('sum')
    #high_comp_df[(high_comp_df.type_s=='twitter') & (high_comp_df.followers_total>0)].head(1)
    low_comp_df['followers_total'] = low_comp_df.groupby('external_link_s').followers_i.transform('sum')
    #low_comp_df[(high_comp_df.type_s=='twitter') & (low_comp_df.followers_total>0)].head(1)
except:
    pass

**Nbr of following per article**

In [46]:
try:
    high_comp_df['following_total'] = high_comp_df.groupby('external_link_s').following_i.transform('sum')
    #high_comp_df[(high_comp_df.type_s=='twitter') & (high_comp_df.following_total>0)].head(1)
    low_comp_df['following_total'] = low_comp_df.groupby('external_link_s').following_i.transform('sum')
    #low_comp_df[(low_comp_df.type_s=='twitter') & (low_comp_df.following_total>0)].head(1)
except:
    pass

**Nbr of mentions per article**

In [47]:
try:
    high_comp_df['mentions_total'] = high_comp_df.groupby('external_link_s').mentions_i.transform('sum')
    #high_comp_df[(high_comp_df.type_s=='twitter') & (high_comp_df.mentions_total>0)].head(1)
    low_comp_df['mentions_total'] = low_comp_df.groupby('external_link_s').mentions_i.transform('sum')
    #low_comp_df[(low_comp_df.type_s=='twitter') & (low_comp_df.mentions_total>0)].head(1)
except:
    pass

**Remove unneeded columns**

In [48]:
unneeded_cols = ['site', 'source_s_lower', 'social_posts']

In [49]:
try:
    high_comp_df.drop(unneeded_cols, axis=1, inplace=True)
    #high_comp_df.head(1)
    low_comp_df.drop(unneeded_cols, axis=1, inplace=True)
    #low_comp_df.head(1)
except:
    pass

In [50]:
print(high_comp_df.shape)
high_comp_df.head(1)

(1967104, 24)


,in_id,type_s,url_s,external_link_s,source_s,timestamp_dt,text_txt_el,retweet_count_l,followers_i,following_i,...,sitename,source_s_new,facebook_posts,tweets,retweets_total,likes_total,comments_total,followers_total,following_total,mentions_total
0,0,twitter,https://twitter.com/iefimerida/status/81525611...,http://www.iefimerida.gr/news/310100/ohi-2016-...,iefimerida,2016-12-31T19:59:39Z,"Οχι, το 2016 δεν ήταν η χειρότερη χρονιά της α...",0.0,172080.0,4278.0,...,iefimerida,iefimerida,0,1,0.0,0.0,0,172080.0,4278.0,25854


### **Make df with posts only from sites accounts**

**Create df with posts from sites only**

In [51]:
try:
    high_comp_posts_from_sites_only_df = high_comp_df[high_comp_df.source_s_new == high_comp_df.sitename]
    low_comp_posts_from_sites_only_df = low_comp_df[low_comp_df.source_s_new == low_comp_df.sitename]
except:
    pass

In [52]:
print(high_comp_posts_from_sites_only_df.shape)
high_comp_posts_from_sites_only_df.head(1)

(1539947, 24)


,in_id,type_s,url_s,external_link_s,source_s,timestamp_dt,text_txt_el,retweet_count_l,followers_i,following_i,...,sitename,source_s_new,facebook_posts,tweets,retweets_total,likes_total,comments_total,followers_total,following_total,mentions_total
0,0,twitter,https://twitter.com/iefimerida/status/81525611...,http://www.iefimerida.gr/news/310100/ohi-2016-...,iefimerida,2016-12-31T19:59:39Z,"Οχι, το 2016 δεν ήταν η χειρότερη χρονιά της α...",0.0,172080.0,4278.0,...,iefimerida,iefimerida,0,1,0.0,0.0,0,172080.0,4278.0,25854


### **ONLY for csvs from iqcsv (τα άλλα έχουν μόνο twitter)** 

**From df with posts from sites only, Create one df with fb posts and one for tweets**

In [53]:
if main_csvs_path == '0_innews_data_sets/iqcsv/':
    try:
        high_sites_fb_df = high_comp_posts_from_sites_only_df[high_comp_posts_from_sites_only_df.type_s=='facebook']
        print(len(high_sites_fb_df))
        high_sites_tw_df = high_comp_posts_from_sites_only_df[high_comp_posts_from_sites_only_df.type_s=='twitter']
        print(len(high_sites_tw_df))
        low_sites_fb_df = low_comp_posts_from_sites_only_df[low_comp_posts_from_sites_only_df.type_s=='facebook']
        print(len(low_sites_fb_df))
        low_sites_tw_df = low_comp_posts_from_sites_only_df[low_comp_posts_from_sites_only_df.type_s=='twitter']
        print(len(low_sites_tw_df))
    except:
        pass

**Remove posts with duplicate values in external_link_s column**

**(keep one post for each article)**

In [54]:
if main_csvs_path == '0_innews_data_sets/iqcsv/':
    try:
        # high
        high_sites_fb_df.drop_duplicates('external_link_s', keep='first', inplace=True)
        #print('fb high')
        #print(high_sites_fb_df.nunique(), '\r')
        high_sites_tw_df.drop_duplicates('external_link_s', keep='first', inplace=True)
        #print('twitter high')
        #print(high_sites_tw_df.nunique())
    except:
        pass
    try:
        # low
        low_sites_fb_df.drop_duplicates('external_link_s', keep='first', inplace=True)
        #print('fb low')
        #print(low_sites_fb_df.nunique(), '\r')
        low_sites_tw_df.drop_duplicates('external_link_s', keep='first', inplace=True)
        #print('twitter low')
        #print(low_sites_tw_df.nunique())
    except:
        pass
else:
    try:
        high_comp_posts_from_sites_only_df.drop_duplicates('external_link_s', keep='first', inplace=True)
        low_comp_posts_from_sites_only_df.drop_duplicates('external_link_s', keep='first', inplace=True)
    except:
        pass   

/tmp/ipykernel_91117/3830863810.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  high_comp_posts_from_sites_only_df.drop_duplicates('external_link_s', keep='first', inplace=True)


In [55]:
#high_sites_tw_df.head(1)

In [56]:
high_comp_posts_from_sites_only_df.head(1)

,in_id,type_s,url_s,external_link_s,source_s,timestamp_dt,text_txt_el,retweet_count_l,followers_i,following_i,...,sitename,source_s_new,facebook_posts,tweets,retweets_total,likes_total,comments_total,followers_total,following_total,mentions_total
0,0,twitter,https://twitter.com/iefimerida/status/81525611...,http://www.iefimerida.gr/news/310100/ohi-2016-...,iefimerida,2016-12-31T19:59:39Z,"Οχι, το 2016 δεν ήταν η χειρότερη χρονιά της α...",0.0,172080.0,4278.0,...,iefimerida,iefimerida,0,1,0.0,0.0,0,172080.0,4278.0,25854


**Remove source_s column**

In [57]:
if main_csvs_path == '0_innews_data_sets/iqcsv/':
    try:
        high_sites_fb_df.drop('source_s', axis=1, inplace=True)
        high_sites_tw_df.drop('source_s', axis=1, inplace=True)
    except:
        pass
    try:
        low_sites_fb_df.drop('source_s', axis=1, inplace=True)
        low_sites_tw_df.drop('source_s', axis=1, inplace=True)
    except:
        pass
else:
    try:
        high_comp_posts_from_sites_only_df.drop('source_s', axis=1, inplace=True)
        low_comp_posts_from_sites_only_df.drop('source_s', axis=1, inplace=True)
    except:
        pass

/tmp/ipykernel_91117/1977936343.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  high_comp_posts_from_sites_only_df.drop('source_s', axis=1, inplace=True)


**Rename columns and Create new dfs**

In [58]:
# facebook posts
try:
    high_sites_fb_df_new = high_sites_fb_df.rename(columns = {
        'in_id':'fb_in_id', 'url_s':'fb_url', 'external_link_s':'article_url',
        'timestamp_dt':'fb_datetime', 'text_txt_el':'fb_text', 'comments_i':'fb_comments',
        'likes_i':'fb_likes', 'full_artcl_text':'article_text', 'sitename':'site',
        'facebook_posts':'total_fb_posts', 'likes_total':'total_fb_likes',
        'comments_total':'total_fb_comments'
    })
    high_sites_fb_df_new = high_sites_fb_df_new[[
        'site', 'article_url', 'article_text', 'fb_in_id', 'fb_url', 'fb_text', 'fb_datetime',
        'fb_likes', 'fb_comments', 'total_fb_posts', 'total_fb_likes', 'total_fb_comments']]
    
    low_sites_fb_df_new = low_sites_fb_df.rename(columns = {
        'in_id':'fb_in_id', 'url_s':'fb_url', 'external_link_s':'article_url',
        'timestamp_dt':'fb_datetime', 'text_txt_el':'fb_text', 'comments_i':'fb_comments',
        'likes_i':'fb_likes', 'full_artcl_text':'article_text', 'sitename':'site',
        'facebook_posts':'total_fb_posts', 'likes_total':'total_fb_likes',
        'comments_total':'total_fb_comments'
    })
    low_sites_fb_df_new = low_sites_fb_df_new[[
        'site', 'article_url', 'article_text', 'fb_in_id', 'fb_url', 'fb_text', 'fb_datetime',
        'fb_likes', 'fb_comments', 'total_fb_posts', 'total_fb_likes', 'total_fb_comments']]
except:
    pass
#high_sites_fb_df_new.head(1)

In [59]:
# twitter
if main_csvs_path == '0_innews_data_sets/iqcsv/':
    try:
        high_sites_tw_df_new = high_sites_tw_df.rename(columns = {
            'in_id':'tw_in_id', 'url_s':'tw_url', 'external_link_s':'article_url',
            'timestamp_dt':'tw_datetime', 'text_txt_el':'tw_text', 'retweet_count_l':'tw_retweets',
            'followers_i':'site_tw_followers', 'following_i':'site_tw_following',
            'mentions_i':'tw_mentions', 'tweets':'total_tweets', 'retweets_total':'total_retweets',
            'followers_total':'total_followers', 'following_total':'total_following',
            'mentions_total':'total_mentions'
        })
        high_sites_tw_df_new = high_sites_tw_df_new[[
            'article_url', 'tw_in_id', 'tw_url', 'tw_text', 'tw_datetime', 'tw_retweets', 'tw_mentions',
            'total_tweets', 'total_retweets', 'total_mentions', 'site_tw_followers', 'site_tw_following',
            'total_followers', 'total_following']]
        
        low_sites_tw_df_new = low_sites_tw_df.rename(columns = {
            'in_id':'tw_in_id', 'url_s':'tw_url', 'external_link_s':'article_url',
            'timestamp_dt':'tw_datetime', 'text_txt_el':'tw_text', 'retweet_count_l':'tw_retweets',
            'followers_i':'site_tw_followers', 'following_i':'site_tw_following',
            'mentions_i':'tw_mentions', 'tweets':'total_tweets', 'retweets_total':'total_retweets',
            'followers_total':'total_followers', 'following_total':'total_following',
            'mentions_total':'total_mentions'})
        low_sites_tw_df_new = low_sites_tw_df_new[[
            'article_url', 'tw_in_id', 'tw_url', 'tw_text', 'tw_datetime', 'tw_retweets',
            'tw_mentions', 'total_tweets', 'total_retweets', 'total_mentions',
            'site_tw_followers', 'site_tw_following', 'total_followers', 'total_following']]
    except:
        pass
else:
    try:
        high_comp_posts_from_sites_only_df = high_comp_posts_from_sites_only_df.rename(columns = {
            'in_id':'tw_in_id', 'url_s':'tw_url', 'external_link_s':'article_url',
            'timestamp_dt':'tw_datetime', 'text_txt_el':'tw_text', 'full_artcl_text':'article_text',
            'sitename':'site', 'retweet_count_l':'tw_retweets', 'followers_i':'site_tw_followers',
            'following_i':'site_tw_following', 'mentions_i':'tw_mentions', 'tweets':'total_tweets',
            'retweets_total':'total_retweets', 'followers_total':'total_followers',
            'following_total':'total_following', 'mentions_total':'total_mentions'
        })
        high_comp_posts_from_sites_only_df = high_comp_posts_from_sites_only_df[[
            'site', 'article_url', 'article_text', 'tw_in_id', 'tw_url', 'tw_text',
            'tw_datetime', 'tw_retweets', 'tw_mentions','total_tweets', 'total_retweets', 'total_mentions',
            'site_tw_followers', 'site_tw_following', 'total_followers', 'total_following']]

        low_comp_posts_from_sites_only_df = low_comp_posts_from_sites_only_df.rename(columns = {
            'in_id':'tw_in_id', 'url_s':'tw_url', 'external_link_s':'article_url',
            'timestamp_dt':'tw_datetime', 'text_txt_el':'tw_text', 'full_artcl_text':'article_text',
            'sitename':'site', 'retweet_count_l':'tw_retweets', 'followers_i':'site_tw_followers',
            'following_i':'site_tw_following', 'mentions_i':'tw_mentions',
            'tweets':'total_tweets', 'retweets_total':'total_retweets',
            'followers_total':'total_followers', 'following_total':'total_following',
            'mentions_total':'total_mentions'
        })
        low_comp_posts_from_sites_only_df = low_comp_posts_from_sites_only_df[[
            'site', 'article_url', 'article_text', 'tw_in_id', 'tw_url', 'tw_text',
            'tw_datetime', 'tw_retweets', 'tw_mentions','total_tweets', 'total_retweets', 'total_mentions',
            'site_tw_followers', 'site_tw_following', 'total_followers', 'total_following']]
    except:
        pass

#high_sites_tw_df_new.head(1)

In [60]:
high_comp_posts_from_sites_only_df.head(1)

,site,article_url,article_text,tw_in_id,tw_url,tw_text,tw_datetime,tw_retweets,tw_mentions,total_tweets,total_retweets,total_mentions,site_tw_followers,site_tw_following,total_followers,total_following
0,iefimerida,http://www.iefimerida.gr/news/310100/ohi-2016-...,"Στο ίντερνετ, το τελευταίο διάστημα ένα σύνθημ...",0,https://twitter.com/iefimerida/status/81525611...,"Οχι, το 2016 δεν ήταν η χειρότερη χρονιά της α...",2016-12-31T19:59:39Z,0.0,25854,1,0.0,25854,172080.0,4278.0,172080.0,4278.0


**Create merged df**

**If csvs are from 2017 or 2022 datasets (twitter only), merged df is twitter df**

In [61]:
if (main_csvs_path == '0_innews_data_sets/iqcsv/'):
    high_sites_merged_df = pd.merge(high_sites_fb_df_new, high_sites_tw_df_new, how='outer', on="article_url")
    high_sites_merged_df.reset_index(drop=True, inplace=True)
    print(f"high sites merged articles: {len(high_sites_merged_df)}")
    high_sites_merged_df.head(1)
    low_sites_merged_df = pd.merge(high_sites_fb_df_new, high_sites_tw_df_new, how='outer', on="article_url")
    low_sites_merged_df.reset_index(drop=True, inplace=True)
    print(f"low sites merged articles: {len(low_sites_merged_df)}")
    high_sites_merged_df.head(1)
else:
    try:
        high_sites_merged_df = high_comp_posts_from_sites_only_df
        print(f"high sites merged articles: {len(high_sites_merged_df)}")
        low_sites_merged_df = low_comp_posts_from_sites_only_df
        print(f"low sites merged articles: {len(low_sites_merged_df)}")
    except:
        pass

high sites merged articles: 1183302


In [62]:
high_sites_merged_df.head(1)

,site,article_url,article_text,tw_in_id,tw_url,tw_text,tw_datetime,tw_retweets,tw_mentions,total_tweets,total_retweets,total_mentions,site_tw_followers,site_tw_following,total_followers,total_following
0,iefimerida,http://www.iefimerida.gr/news/310100/ohi-2016-...,"Στο ίντερνετ, το τελευταίο διάστημα ένα σύνθημ...",0,https://twitter.com/iefimerida/status/81525611...,"Οχι, το 2016 δεν ήταν η χειρότερη χρονιά της α...",2016-12-31T19:59:39Z,0.0,25854,1,0.0,25854,172080.0,4278.0,172080.0,4278.0


## **Final datasets**

In [63]:
try:
    print('high sites:', high_sites_merged_df.site.nunique())
    print('high articles with social posts:', len(high_sites_merged_df))
    print('low sites:', low_sites_merged_df.site.nunique())
    print('low articles with social posts:', len(low_sites_merged_df))
except:
    pass

high sites: 47
high articles with social posts: 1183302


## Export csvs

In [64]:
if main_csvs_path == '0_innews_data_sets/iqcsv/':
    filename = "2016"
    try:
        high_sites_fb_tw_df = high_sites_merged_df[(~high_sites_merged_df.total_fb_posts.isna())\
                                                   & (~high_sites_merged_df.total_tweets.isna())].copy()
        print(high_sites_merged_df.shape, high_sites_fb_tw_df.shape)
        low_sites_fb_tw_df = low_sites_merged_df[(~low_sites_merged_df.total_fb_posts.isna())\
                                                   & (~low_sites_merged_df.total_tweets.isna())].copy()
        print(low_sites_merged_df.shape, low_sites_fb_tw_df.shape)
    except:
        pass
else:
    if main_csvs_path == '0_innews_data_sets/exports_2017/':
        filename = "2017-2018"
    elif main_csvs_path == '0_innews_data_sets/exports_2022/':
        filename = "2022-2023"
    try:
        high_sites_fb_tw_df = high_sites_merged_df.copy()
        print(high_sites_fb_tw_df.shape)
        low_sites_fb_tw_df = low_sites_merged_df.copy()
        print(low_sites_fb_tw_df.shape)
    except:
        pass
        

(1183302, 16)


In [65]:
# high sites nan check
try:
    high_sites_fb_tw_df['title'] = \
    high_sites_fb_tw_df['tw_text'].str.replace('\n', ' ') \
    .str.replace('http[\:a-zA-Z\/\.0-9]*|\#[a-zA-Z]*|\s{2,}', '', regex=True)
    high_sites_fb_tw_df.insert(2, 'article_title', high_sites_fb_tw_df.pop('title'))
    print(high_sites_fb_tw_df.isna().sum())
    high_sites_fb_tw_df.head()
except:
    pass

site                 0
article_url          0
article_title        0
article_text         0
tw_in_id             0
tw_url               0
tw_text              0
tw_datetime          0
tw_retweets          0
tw_mentions          0
total_tweets         0
total_retweets       0
total_mentions       0
site_tw_followers    0
site_tw_following    0
total_followers      0
total_following      0
dtype: int64


In [66]:
# low sites nan check
try:
    low_sites_fb_tw_df['title'] = \
    low_sites_fb_tw_df['tw_text'].str.replace('\n', ' ') \
    .str.replace('http[\:a-zA-Z\/\.0-9]*|\#[a-zA-Z]*|\s{2,}', '', regex=True)
    low_sites_fb_tw_df.insert(2, 'article_title', low_sites_fb_tw_df.pop('title'))
    print(low_sites_fb_tw_df.isna().sum())
    low_sites_fb_tw_df.head()
except:
    pass

In [67]:
high_sites_fb_tw_df.columns

Index(['site', 'article_url', 'article_title', 'article_text', 'tw_in_id',
       'tw_url', 'tw_text', 'tw_datetime', 'tw_retweets', 'tw_mentions',
       'total_tweets', 'total_retweets', 'total_mentions', 'site_tw_followers',
       'site_tw_following', 'total_followers', 'total_following'],
      dtype='object')

**Clean \n from text (article, tw, fb)**

In [68]:
for col_name in ['article_text', 'fb_text', 'tw_text']:
    try:
        high_sites_fb_tw_df[col_name] = \
        high_sites_fb_tw_df[col_name].str.replace('\xa0', ' ').str.replace('\r', ' ')\
        .str.replace('\n', ' ').str.replace('\t', ' ').str.replace('    ', ' ')\
        .str.replace('   ', ' ').str.replace('  ', ' ')
        low_sites_fb_tw_df[col_name] = \
        low_sites_fb_tw_df[col_name].str.replace('\xa0', ' ').str.replace('\r', ' ')\
        .str.replace('\n', ' ').str.replace('\t', ' ').str.replace('    ', ' ')\
        .str.replace('   ', ' ').str.replace('  ', ' ')
    except:
        pass

### **Save csvs**

In [69]:
try:
    high_sites_fb_tw_df.to_csv('0_csvs_for_IQ_feed/high_sites_' + filename + '.csv', index=False)
    low_sites_fb_tw_df.to_csv('0_csvs_for_IQ_feed/low_sites_' + filename + '.csv', index=False)
except:
    pass

In [71]:
high_sites_df = pd.read_csv("0_csvs_for_IQ_feed/high_sites_2017-2018.csv")
high_sites_df

,site,article_url,article_title,article_text,tw_in_id,tw_url,tw_text,tw_datetime,tw_retweets,tw_mentions,total_tweets,total_retweets,total_mentions,site_tw_followers,site_tw_following,total_followers,total_following
0,iefimerida,http://www.iefimerida.gr/news/310100/ohi-2016-...,"Οχι, το 2016 δεν ήταν η χειρότερη χρονιά της α...","Στο ίντερνετ, το τελευταίο διάστημα ένα σύνθημ...",0,https://twitter.com/iefimerida/status/81525611...,"Οχι, το 2016 δεν ήταν η χειρότερη χρονιά της α...",2016-12-31T19:59:39Z,0.0,25854.0,1,0.0,25854.0,172080.0,4278.0,172080.0,4278.0
1,iefimerida,http://www.iefimerida.gr/news/310407/ypoyrgeio...,Το υπουργείο Οικονομίας διέψευσε τη «μεγάλη φυ...,"Το υπουργείο Οικονομίας και Ανάπτυξης, με άτυπ...",1,https://twitter.com/iefimerida/status/81526386...,Το υπουργείο Οικονομίας διέψευσε τη «μεγάλη φυ...,2016-12-31T20:30:26Z,0.0,25854.0,2,0.0,25854.0,172080.0,4278.0,311703.0,4278.0
2,iefimerida,http://www.iefimerida.gr/news/310404/minyma-mi...,Μήνυμα Μητσοτάκη για το νέο έτος: «Είναι η ώρα...,"«Το 2016 φεύγει, αφήνοντας πίσω του ένα αίσθημ...",2,https://twitter.com/iefimerida/status/81525431...,Μήνυμα Μητσοτάκη για το νέο έτος: «Είναι η ώρα...,2016-12-31T19:52:29Z,0.0,25854.0,1,0.0,25854.0,172080.0,4278.0,172080.0,4278.0
3,iefimerida,http://www.iefimerida.gr/news/310408/me-ilio-k...,Με ήλιο και κρύο «μπαίνει» η νέα χρονιά -Βελτί...,"Με ήλιο, κρύο και χιόνια ξεκινάει η χρονιά. Αν...",3,https://twitter.com/iefimerida/status/81528249...,Με ήλιο και κρύο «μπαίνει» η νέα χρονιά -Βελτί...,2016-12-31T21:44:29Z,0.0,25854.0,1,0.0,25854.0,172080.0,4278.0,172080.0,4278.0
4,iefimerida,http://www.iefimerida.gr/news/310411/akatallil...,Ακατάλληλο για κατανάλωση το νερό στο Μεσολόγγ...,Ακατάλληλο για κατανάλωση και χρήση κρίθηκε το...,4,https://twitter.com/iefimerida/status/81527680...,Ακατάλληλο για κατανάλωση το νερό στο Μεσολόγγ...,2016-12-31T21:21:52Z,0.0,25854.0,1,0.0,25854.0,172080.0,4278.0,172080.0,4278.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1183297,presspublica,https://www.presspublica.gr/toyrkia-vinteo-me-...,Τουρκία: Βίντεο με άνδρες που φέρεται ότι μετα...,"Άνδρες που μεταφέρουν βαλίτσες, μέσα στις οποί...",1967480,https://twitter.com/PressPublica/status/107981...,Τουρκία: Βίντεο με άνδρες που φέρεται ότι μετα...,2018-12-31T21:17:45Z,0.0,0.0,1,0.0,0.0,1396.0,432.0,1396.0,432.0
1183298,presspublica,https://www.presspublica.gr/st-theodorakis-to-...,Στ. Θεοδωράκης: Το 2019 να σκεφτούμε και να πρ...,«Το 2019 μπορεί να είναι μία αληθινά νέα χρονι...,1967481,https://twitter.com/PressPublica/status/107982...,Στ. Θεοδωράκης: Το 2019 να σκεφτούμε και να πρ...,2018-12-31T21:27:40Z,0.0,0.0,1,0.0,0.0,1396.0,432.0,1396.0,432.0
1183299,presspublica,https://www.presspublica.gr/mitsotakis-i-chora...,Μητσοτάκης: Η χώρα δεν χρειάζεται άλλους διχασ...,«Το γύρισμα του χρόνου μάς οδηγεί και σε ένα γ...,1967482,https://twitter.com/PressPublica/status/107982...,Μητσοτάκης: Η χώρα δεν χρειάζεται άλλους διχασ...,2018-12-31T21:49:19Z,0.0,0.0,1,0.0,0.0,1396.0,432.0,1396.0,432.0
1183300,presspublica,https://www.presspublica.gr/tramp-argi-i-apoch...,Τραμπ: «Αργή» η αποχώρηση των Αμερικανών στρατ...,Ο πρόεδρος των ΗΠΑ Ντόναλντ Τραμπ επιβεβαίωσε ...,1967483,https://twitter.com/PressPublica/status/107982...,Τραμπ: «Αργή» η αποχώρηση των Αμερικανών στρατ...,2018-12-31T21:52:09Z,0.0,0.0,1,0.0,0.0,1396.0,432.0,1396.0,432.0


## **Top articles based on engagement**

**High quality sites**

In [78]:
if main_csvs_path == '0_innews_data_sets/iqcsv/':
    high_top_75_df = \
    high_sites_df[(high_sites_df['total_fb_posts'].ge(high_sites_df['total_fb_posts'].quantile(q=.75))) &
                        (high_sites_df['total_fb_likes'].ge(high_sites_df['total_fb_likes'].quantile(q=.75))) &
                        (high_sites_df['total_fb_comments'].ge(high_sites_df['total_fb_comments'].quantile(q=.75))) &
                        (high_sites_df['total_tweets'].ge(high_sites_df['total_tweets'].quantile(q=.75))) & 
                        (high_sites_df['total_retweets'].ge(high_sites_df['total_retweets'].quantile(q=.75)))
                        ]

    print(high_top_75_df.site.nunique())
    print(high_top_75_df.site.unique())
    print(high_top_75_df.shape)
elif main_csvs_path == '0_innews_data_sets/exports_2022/':
    high_top_near_mean_df = \
    high_sites_df[(high_sites_df['total_tweets'].ge(0.09)) & 
                  (high_sites_df['total_retweets'].ge(0.07))]
    print(high_top_near_mean_df.site.nunique())
    print(high_top_near_mean_df.site.unique())
    print(high_top_near_mean_df.shape)
elif main_csvs_path == '0_innews_data_sets/exports_2017/':
    high_top_98_df = \
    high_sites_df[(high_sites_df['total_tweets'].ge(high_sites_df['total_tweets'].quantile(q=.98))) & 
                  (high_sites_df['total_retweets'].ge(high_sites_df['total_retweets'].quantile(q=.98)))]
    print(high_top_98_df.site.nunique())
    print(high_top_98_df.site.unique())
    print(high_top_98_df.shape)   

30
['iefimerida' 'tvxs' 'thetoc' 'stokokkino' 'skai' 'tovima' 'ert' 'zougla'
 'enikonomia' 'athensvoice' 'star' 'parapolitika' 'antinews' 'enikos'
 'news247' 'balla' 'infokids' 'cyclades24' 'avgi' 'naftemporiki'
 'paraskhnio' 'efsyn' 'kathimerini' 'amna' 'protagon' 'newspao' 'left'
 'real' 'ipaideia' 'newpost']
(4152, 17)


In [79]:
if main_csvs_path == '0_innews_data_sets/iqcsv/':
    high_top_75_df.to_csv('0_csvs_for_IQ_feed/high_sites_'+filename+'_top_75.csv', index=False)
elif main_csvs_path == '0_innews_data_sets/exports_2022/':
    high_top_near_mean_df.to_csv('0_csvs_for_IQ_feed/high_sites_'+filename+'_near_mean.csv', index=False)
elif main_csvs_path == '0_innews_data_sets/exports_2017/':
    high_top_98_df.to_csv('0_csvs_for_IQ_feed/high_sites_'+filename+'_top_98.csv', index=False)

**Low quality sites**

In [ ]:
if main_csvs_path == '0_innews_data_sets/iqcsv/':
    low_top_66_df = \
    low_sites_df[(low_sites_df['total_fb_posts'].ge(low_sites_df['total_fb_posts'].quantile(q=.66))) &
                        (low_sites_df['total_fb_likes'].ge(low_sites_df['total_fb_likes'].quantile(q=.66))) &
                        (low_sites_df['total_fb_comments'].ge(low_sites_df['total_fb_comments'].quantile(q=.66))) &
                        (low_sites_df['total_tweets'].ge(low_sites_df['total_tweets'].quantile(q=.66))) & 
                        (low_sites_df['total_retweets'].ge(low_sites_df['total_retweets'].quantile(q=.66)))
                        ]

    print(low_top_66_df.site.nunique())
    print(low_top_66_df.site.unique())
    print(low_top_66_df.shape)

In [ ]:
if main_csvs_path == '0_innews_data_sets/iqcsv/':
    low_top_66_df.to_csv('0_csvs_for_IQ_feed/high_sites_'+filename+'_top_66.csv', index=False)